In [1]:
import os
import json
from groq import Groq

In [2]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
model = "llama3-70b-8192"

In [3]:
def get_weather(city : str):
    if city == "Madrid":
        return 35
    elif city == "San Francisco":
        return 18
    elif city == "Paris":
        return 20
    else:
        return 15

In [4]:
messages = [
            {"role" : "system", "content" : """You are a helpful assistant.""" },
            {"role" : "user", "content" : "What is the weather in Paris, Tokyo and Madrid?"},
]

tools = [
    { "type" : "function",
      "function" : {
                    "name" : "get_weather",
                    "description" : "Returns the weather in the given city in degrees celsius",
                    "parameters" : {
                         "type" : "object",
                         "properties" : {
                                  "city" : {"type" : "string",
                                            "description" : "The name of the city"}
                         },
                         "required": ["city"],
                    }
      }
    }
]

response = client.chat.completions.create(model = model,
                                          messages=messages,
                                         tools = tools,
                                         tool_choice="auto",
                                         max_tokens=4096)
response_message = response.choices[0].message

In [5]:
import json

In [6]:
tool_calls = response_message.tool_calls

messages.append(
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": tool_call.id,
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                },
                "type": tool_call.type,
            }
            for tool_call in tool_calls
        ],
    }
)

available_functions = {
    "get_weather": get_weather,
}
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    # Note how we create a separate tool call message for each tool call
    # the model is able to discern the tool call result through the tool_call_id
    messages.append(
        {
            "role": "tool",
            "content": json.dumps(function_response),
            "tool_call_id": tool_call.id,
        }
    )

print(json.dumps(messages, indent=2))

[
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": "What is the weather in Paris, Tokyo and Madrid?"
  },
  {
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_4q9h",
        "function": {
          "name": "get_weather",
          "arguments": "{\"city\":\"Paris\"}"
        },
        "type": "function"
      },
      {
        "id": "call_7x6z",
        "function": {
          "name": "get_weather",
          "arguments": "{\"city\":\"Tokyo\"}"
        },
        "type": "function"
      },
      {
        "id": "call_0yzr",
        "function": {
          "name": "get_weather",
          "arguments": "{\"city\":\"Madrid\"}"
        },
        "type": "function"
      }
    ]
  },
  {
    "role": "tool",
    "content": "20",
    "tool_call_id": "call_4q9h"
  },
  {
    "role": "tool",
    "content": "15",
    "tool_call_id": "call_7x6z"
  },
  {
    "role": "tool",
    "content": "35",
    "tool

In [7]:
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
)

print(response.choices[0].message.content)

The weather in Paris is 20 degrees celsius, in Tokyo it is 15 degrees celsius, and in Madrid it is 35 degrees celsius.
